In [ ]:
from grid.interface import *

In [ ]:
refresh()

In [ ]:
vast.launch('cuda_max_good >= 11.0 & rentable & gpu_name == "RTX 2080 Ti" & machine_id != 335', default=True)

In [ ]:
vast.stats().set_index('date').resample('15min').mean().plot()

In [ ]:
jittens.manage.tails('output/pavlov/*/logs.0.txt', count=500, lineglob='*Approx*')

In [ ]:
dead = set(jittens.machines.machines()) - {j.machine for j in jittens.jobs.jobs('active').values()}
for d in dead:
    print(jittens.machines.machines()[d].connection_kwargs['port'])

In [ ]:
m = jittens.jobs.jobs()['2021-02-19 20-28-28 wry-disk'].machine
jittens.machines.machines()[m].connection_kwargs['port']

In [ ]:
for j, i in jittens.jobs.jobs('active').items():
    try:
        print(j, '\t\t', i.machine, jittens.machines.machines()[i.machine].connection_kwargs['port'])
    except:
        pass